In [1]:
import numpy as np
import pandas as pd

In [2]:
weather = pd.read_csv('weather_by_station.csv', index_col='date', parse_dates=True)

In [3]:
fb = pd.read_csv('fb_2018.csv', index_col='date', parse_dates=True).assign(
    trading_volume=lambda x: pd.cut(x.volume, bins=3, labels=['low', 'med', 'high'])
)
fb.head()

,open,high,low,close,volume,trading_volume
date,,,,,,
2018-01-02,177.68,181.58,177.5500,181.42,18151903,low
2018-01-03,181.88,184.78,181.3300,184.67,16886563,low
2018-01-04,184.90,186.21,184.0996,184.33,13880896,low
2018-01-05,185.59,186.90,184.9300,186.85,13574535,low
2018-01-08,187.20,188.90,186.3300,188.28,17994726,low


Before we dive into any calculations, let's make sure pandas won't put things in scientific notation. We will modify how floats are formatted for displaying. The format we will apply is .2f, which will provide the float with 2 digits after the decimal point:

In [4]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

Summarizing DataFrames with agg():

In [5]:
fb.agg({
    'open': np.mean, 
    'high': np.max, 
    'low': np.min, 
    'close': np.mean, 
    'volume': np.sum})

/tmp/ipykernel_3253/4107537763.py:1: FutureWarning: The provided callable <function mean at 0x7faeb81b0dc0> is currently using Series.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  fb.agg({
/tmp/ipykernel_3253/4107537763.py:1: FutureWarning: The provided callable <function amax at 0x7faeb81b01f0> is currently using Series.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  fb.agg({
/tmp/ipykernel_3253/4107537763.py:1: FutureWarning: The provided callable <function amin at 0x7faeb81b03a0> is currently using Series.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  fb.agg({
/tmp/ipykernel_3253/4107537763.py:1: FutureWarning: The provided callable <function sum at 0x7faeb819f910> is currently using Series.sum. In a future ve

open            171.45
high            218.62
low             123.02
close           171.51
volume   6949682394.00
dtype: float64

We can use this to find the total snowfall and precipitation recorded in Central Park in 2018:

In [6]:
weather.query(
    'station == "GHCND:USW00094728"'
).pivot(columns='datatype', values='value')[['SNOW', 'PRCP']].sum()

datatype
SNOW   1007.00
PRCP   1665.30
dtype: float64

This is equivalent to passing 'sum' to agg():

In [7]:
weather.query(
    'station == "GHCND:USW00094728"'
).pivot(columns='datatype', values='value')[['SNOW', 'PRCP']].agg('sum')

datatype
SNOW   1007.00
PRCP   1665.30
dtype: float64

Note that we aren't limited to providing a single aggregation per column. We can pass a list, and we will get a dataframe back instead of a series. nan values are placed where we don't have a calculation result to display:

In [8]:
fb.agg({
    'open': 'mean',
    'high': ['min', 'max'],
    'low': ['min', 'max'],
    'close': 'mean'
})

,open,high,low,close
mean,171.45,NaN,NaN,171.51
min,NaN,129.74,123.02,NaN
max,NaN,218.62,214.27,NaN


Often we won't want to aggregate on the entire dataframe, but on groups within it. For this purpose, we can run groupby() before the aggregation. If we group by the trading_volume column, we will get a row for each of the values it takes on:

In [9]:
fb.groupby('trading_volume').mean()

/tmp/ipykernel_3253/46027550.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  fb.groupby('trading_volume').mean()


,open,high,low,close,volume
trading_volume,,,,,
low,171.36,173.46,169.31,171.43,24547207.71
med,175.82,179.42,172.11,175.14,79072559.12
high,167.73,170.48,161.57,168.16,141924023.33


After we run the groupby(), we can still select columns for aggregation:

In [10]:
fb.groupby('trading_volume')['close'].agg(['min', 'max', 'mean'])

/tmp/ipykernel_3253/3607524933.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  fb.groupby('trading_volume')['close'].agg(['min', 'max', 'mean'])


,min,max,mean
trading_volume,,,
low,124.06,214.67,171.43
med,152.22,217.50,175.14
high,160.06,176.26,168.16


We can still provide a dictionary specifying the aggregations to perform, but passing a list for a column will result in a hierarchical index for the columns:

In [11]:
fb_agg = fb.groupby('trading_volume').agg({
    'open': 'mean',
    'high': ['min', 'max'],
    'low': ['min', 'max'],
    'close': 'mean'
})
fb_agg

/tmp/ipykernel_3253/2667628102.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  fb_agg = fb.groupby('trading_volume').agg({


open   high           low         close
                 mean    min    max    min    max   mean
trading_volume                                          
low            171.36 129.74 216.20 123.02 212.60 171.43
med            175.82 162.85 218.62 150.75 214.27 175.14
high           167.73 161.10 180.13 149.02 173.75 168.16

In [12]:
fb_agg.columns

MultiIndex([( 'open', 'mean'),
            ( 'high',  'min'),
            ( 'high',  'max'),
            (  'low',  'min'),
            (  'low',  'max'),
            ('close', 'mean')],
           )

Using a list comprehension, we can join the levels (in a tuple) with an _ at each iteration:

In [13]:
fb_agg.columns = ['_'.join(col_agg) for col_agg in fb_agg.columns]
fb_agg.head()

,open_mean,high_min,high_max,low_min,low_max,close_mean
trading_volume,,,,,,
low,171.36,129.74,216.20,123.02,212.60,171.43
med,175.82,162.85,218.62,150.75,214.27,175.14
high,167.73,161.10,180.13,149.02,173.75,168.16


We can group on datetimes despite them being in the index if we use a Grouper:

In [14]:
print(weather.keys())


Index(['datatype', 'station', 'value', 'station_name'], dtype='object')


In [15]:
print(weather.head())

           datatype            station  value                    station_name
date                                                                         
2018-01-01     PRCP  GHCND:US1CTFR0039   0.00           STAMFORD 4.2 S, CT US
2018-01-01     PRCP  GHCND:US1NJBG0015   0.00  NORTH ARLINGTON 0.7 WNW, NJ US
2018-01-01     SNOW  GHCND:US1NJBG0015   0.00  NORTH ARLINGTON 0.7 WNW, NJ US
2018-01-01     PRCP  GHCND:US1NJBG0017   0.00        GLEN ROCK 0.7 SSE, NJ US
2018-01-01     SNOW  GHCND:US1NJBG0017   0.00        GLEN ROCK 0.7 SSE, NJ US


In [16]:
print(weather.columns)

Index(['datatype', 'station', 'value', 'station_name'], dtype='object')


In [17]:
print(weather.dtypes)

datatype         object
station          object
value           float64
station_name     object
dtype: object


This Grouper can be one of many group by values. Here, we find the quarterly total precipitation per station:

we can use filter() to exclude some groups from aggregation. Here, we only keep groups with 'NY' in the group's name attribute, which is the station ID in this case:

In [18]:
weather[weather['station'].str.contains('NY', na=False)]

,datatype,station,value,station_name
date,,,,
2018-01-01,PRCP,GHCND:US1NYKN0025,0.00,"BROOKLYN 3.1 NW, NY US"
2018-01-01,SNOW,GHCND:US1NYKN0025,0.00,"BROOKLYN 3.1 NW, NY US"
2018-01-01,WESF,GHCND:US1NYKN0025,0.00,"BROOKLYN 3.1 NW, NY US"
2018-01-01,PRCP,GHCND:US1NYNS0007,0.00,"FLORAL PARK 0.4 W, NY US"
2018-01-01,SNOW,GHCND:US1NYNS0007,0.00,"FLORAL PARK 0.4 W, NY US"
...,...,...,...,...
2018-12-31,PRCP,GHCND:US1NYWC0018,0.00,"ARMONK 0.3 SE, NY US"
2018-12-31,SNOW,GHCND:US1NYWC0018,0.00,"ARMONK 0.3 SE, NY US"
2018-12-31,SNWD,GHCND:US1NYWC0018,0.00,"ARMONK 0.3 SE, NY US"


In [19]:
weather[weather['datatype'] == "SNOW"]

,datatype,station,value,station_name
date,,,,
2018-01-01,SNOW,GHCND:US1NJBG0015,0.00,"NORTH ARLINGTON 0.7 WNW, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0017,0.00,"GLEN ROCK 0.7 SSE, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0018,0.00,"PALISADES PARK 0.2 WNW, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0023,0.00,"OAKLAND 0.9 SSE, NJ US"
2018-01-01,SNOW,GHCND:US1NJBG0039,0.00,"RIVER EDGE 0.4 NNE, NJ US"
...,...,...,...,...
2018-12-31,SNOW,GHCND:USC00308577,0.00,"SYOSSET, NY US"
2018-12-31,SNOW,GHCND:USW00014732,0.00,"LA GUARDIA AIRPORT, NY US"
2018-12-31,SNOW,GHCND:USW00014734,0.00,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US"


In [20]:
result = (
    weather.groupby('station')
    .filter(lambda x: 'NY' in x.name)
    .query('datatype == "SNOW"')
    .groupby('station_name')
    .sum(numeric_only=True)  # Only aggregate numeric columns
    .squeeze()
)

In [21]:
print(result)

station_name
ALBERTSON 0.2 SSE, NY US         1087.00
AMITYVILLE 0.1 WSW, NY US         434.00
AMITYVILLE 0.6 NNE, NY US        1072.00
ARMONK 0.3 SE, NY US             1504.00
BROOKLYN 3.1 NW, NY US            305.00
CENTERPORT 0.9 SW, NY US          799.00
ELMSFORD 0.8 SSW, NY US           863.00
FLORAL PARK 0.4 W, NY US         1015.00
HICKSVILLE 1.3 ENE, NY US         716.00
JACKSON HEIGHTS 0.3 WSW, NY US    107.00
LOCUST VALLEY 0.3 E, NY US          0.00
LYNBROOK 0.3 NW, NY US            325.00
MASSAPEQUA 0.9 SSW, NY US          41.00
MIDDLE VILLAGE 0.5 SW, NY US     1249.00
NEW HYDE PARK 1.6 NE, NY US         0.00
NEW YORK 8.8 N, NY US               0.00
NORTH WANTAGH 0.4 WSW, NY US      471.00
PLAINEDGE 0.4 WSW, NY US          610.00
PLAINVIEW 0.4 ENE, NY US         1360.00
SADDLE ROCK 3.4 WSW, NY US        707.00
STATEN ISLAND 1.4 SE, NY US       936.00
STATEN ISLAND 4.5 SSE, NY US       89.00
SYOSSET 2.0 SSW, NY US           1039.00
VALLEY STREAM 0.6 SE, NY US       898.00
WAN

Let's see which months have the most precipitation. First, we need to group by day and average the precipitation across the stations. Then we can group by month and sum the resulting precipitation. We use nlargest() to give the 5 months with the most precipitation:

In [22]:
import pandas as pd

# Example: Add a fake date column starting from a given date
weather['date'] = pd.date_range(start='2024-01-01', periods=len(weather), freq='D')
weather.set_index('date', inplace=True)

# Filter, group, and aggregate
result = (
    weather.query('datatype == "PRCP"')
    .select_dtypes(include=['number'])  # Keep only numeric columns
    .groupby(pd.Grouper(freq='D'))  # Group by day
    .mean()  # Daily mean
    .groupby(pd.Grouper(freq='M'))  # Group by month
    .sum()  # Monthly sum
)

# Find the largest values in the 'value' column
top_5 = result['value'].nlargest(5)
print(top_5)

date
2185-12-31   1469.70
2186-01-31   1223.90
2221-04-30   1142.50
2090-09-30   1016.10
2091-05-31   1004.00
Name: value, dtype: float64


 Snow will count towards precipitation, but that doesn't explain why summer months are higher than April. Let's look for days that accounted for a large percentage of the precipitation in a given month.

In order to do so, we need to calculate the average daily precipitation across stations and then find the total per month. This will be the denominator. However, in order to divide the daily values by the total for their month, we will need a Series of equal dimensions. This means we will need to use transform()

In [23]:
weather['date'] = pd.date_range(start='2018-01-28', periods=len(weather), freq='D')
weather.set_index('date', inplace=True)

In [24]:
# Assuming 'weather' is your DataFrame and it does not have a 'date' column
weather['date'] = pd.date_range(start='2018-01-28', periods=len(weather), freq='D')  # Create 'date' column with a date range
weather.set_index('date', inplace=True)  # Set 'date' as the index (you don't need pd.to_datetime here)

# Now filter, rename, and aggregate the data
result = (
    weather.query('datatype == "PRCP"')  # Filter rows where datatype == "PRCP"
    .rename(columns={'value': 'prcp'})  # Rename 'value' column to 'prcp'
    .select_dtypes(include=['number'])  # Keep only numeric columns for aggregation
    .groupby(pd.Grouper(freq='D'))  # Group by day
    .mean()  # Calculate daily mean
    .groupby(pd.Grouper(freq='M'))  # Group by month
    .sum()  # Calculate monthly sum
)

# Slice the date range and print the result
result = result.loc['2018-01-28':'2018-02-03']
print(result)

            prcp
date            
2018-01-31  0.00


In [25]:
# Create the 'date' column with a date range (replace this step if you already have the 'date' column)
weather['date'] = pd.date_range(start='2018-01-28', periods=len(weather), freq='D')

# Set 'date' as the index
weather.set_index('date', inplace=True)

# Filter and rename columns
result = (
    weather.query('datatype == "PRCP"')  # Filter rows where datatype == "PRCP"
    .rename(columns={'value': 'prcp'})  # Rename 'value' column to 'prcp'
    .select_dtypes(include=['number'])  # Keep only numeric columns for aggregation
)

# Apply the 'transform()' function to calculate monthly sums, preserving the row count
result['monthly_sum'] = (
    result.groupby(pd.Grouper(freq='M'))  # Group by month
    .transform(np.sum)  # Apply the sum transformation per group, preserving shape
)

# Filter results for the desired date range
result = result.loc['2018-01-28':'2018-02-03']

# Print the results
print(result)

            prcp  monthly_sum
date                         
2018-01-28  0.00         0.00
2018-01-29  0.00         0.00
2018-01-31  0.00         0.00
2018-02-03  0.00         0.00


/tmp/ipykernel_3253/2387777430.py:17: FutureWarning: The provided callable <function sum at 0x7faeb819f910> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  .transform(np.sum)  # Apply the sum transformation per group, preserving shape


In [26]:
# Sample DataFrame creation (ensure you have the correct columns)
weather['date'] = pd.date_range(start='2018-01-28', periods=len(weather), freq='D')
weather.set_index('date', inplace=True)

# Filter and rename columns
result = (
    weather.query('datatype == "PRCP"')  # Filter rows where datatype == "PRCP"
    .rename(columns={'value': 'prcp'})  # Rename 'value' column to 'prcp'
    .select_dtypes(include=['number'])  # Keep only numeric columns for aggregation
)

# Check the shape of the filtered DataFrame
print("Shape after filtering and renaming:", result.shape)

# Group by month and check the shape of the grouped DataFrame
grouped = result.groupby(pd.Grouper(freq='M'))
print("Number of groups:", len(grouped))

# Apply the transformation and check the result
monthly_sum_check = grouped.transform(np.sum)
print("Shape after transform:", monthly_sum_check.shape)

# Verify if transform result has the same shape as original
print("Shape of original result vs. transformed result:", result.shape, monthly_sum_check.shape)

Shape after filtering and renaming: (28129, 1)
Number of groups: 2638
Shape after transform: (28129, 1)
Shape of original result vs. transformed result: (28129, 1) (28129, 1)


/tmp/ipykernel_3253/1777327004.py:20: FutureWarning: The provided callable <function sum at 0x7faeb819f910> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  monthly_sum_check = grouped.transform(np.sum)


In [27]:
# transform() can be used on dataframes as well. We can use it to easily standardize the data:
fb[['open', 'high', 'low', 'close']].transform(
    lambda x: (x - x.mean()).div(x.std())
).head()

,open,high,low,close
date,,,,
2018-01-02,0.32,0.41,0.41,0.50
2018-01-03,0.53,0.57,0.60,0.66
2018-01-04,0.68,0.65,0.74,0.64
2018-01-05,0.72,0.68,0.78,0.77
2018-01-08,0.80,0.79,0.85,0.84


Pivot tables and crosstabs
We saw pivots in chapter 3; however, we weren't able to provide any aggregations. With pivot_table(), we get the mean by default as the aggfunc. In its simplest form, we provide a column to place along the columns:

In [28]:
fb.pivot_table(columns='trading_volume')

trading_volume,low,med,high
close,171.43,175.14,168.16
high,173.46,179.42,170.48
low,169.31,172.11,161.57
open,171.36,175.82,167.73
volume,24547207.71,79072559.12,141924023.33


By placing the trading volume in the index, we get the aggregation from the first example in the group by section above:

In [29]:
fb.pivot_table(index='trading_volume')

,close,high,low,open,volume
trading_volume,,,,,
low,171.43,173.46,169.31,171.36,24547207.71
med,175.14,179.42,172.11,175.82,79072559.12
high,168.16,170.48,161.57,167.73,141924023.33


With pivot(), we also weren't able to handle multi-level indices or indices with repeated values. For this reason we haven't been able to put the weather data in the wide format. The pivot_table() method solves this issue:

In [30]:
weather.reset_index().pivot_table(
    index=['date', 'station', 'station_name'], 
    columns='datatype', 
    values='value',
    aggfunc='median'
).reset_index().tail()

datatype,date,station,station_name,AWND,DAPR,MDPR,PGTM,PRCP,SNOW,SNWD,...,WSF5,WT01,WT02,WT03,WT04,WT05,WT06,WT08,WT09,WT11
80251,2237-10-18,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80252,2237-10-19,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80253,2237-10-20,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80254,2237-10-21,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80255,2237-10-22,GHCND:USW00094789,"JFK INTERNATIONAL AIRPORT, NY US",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can use the pd.crosstab() function to create a frequency table. For example, if we want to see how many low-, medium-, and high-volume trading days Facebook stock had each month, we can use crosstab:

In [31]:
pd.crosstab(
    index=fb.trading_volume,
    columns=fb.index.month,
    colnames=['month'] # name the columns index
)

month,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,20,19,15,20,22,21,18,23,19,23,21,19
med,1,0,4,1,0,0,2,0,0,0,0,0
high,0,0,2,0,0,0,1,0,0,0,0,0


We can normalize with the row or column totals with the normalize parameter. This shows percentage of the total:

In [32]:
pd.crosstab(
    index=fb.trading_volume,
    columns=fb.index.month,
    colnames=['month'],
    normalize='columns'
)

month,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,0.95,1.00,0.71,0.95,1.00,1.00,0.86,1.00,1.00,1.00,1.00,1.00
med,0.05,0.00,0.19,0.05,0.00,0.00,0.10,0.00,0.00,0.00,0.00,0.00
high,0.00,0.00,0.10,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00


If we want to perform a calculation other than counting the frequency, we can pass the column to run the calculation on to values and the function to use to aggfunc:

In [33]:
pd.crosstab(
    index=fb.trading_volume,
    columns=fb.index.month,
    colnames=['month'],
    values=fb.close,
    aggfunc=np.mean
)

/tmp/ipykernel_3253/3586945560.py:1: FutureWarning: The provided callable <function mean at 0x7faeb81b0dc0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  pd.crosstab(


month,1,2,3,4,5,6,7,8,9,10,11,12
trading_volume,,,,,,,,,,,,
low,185.24,180.27,177.07,163.29,182.93,195.27,201.92,177.49,164.38,154.19,141.64,137.16
med,179.37,NaN,164.76,174.16,NaN,NaN,194.28,NaN,NaN,NaN,NaN,NaN
high,NaN,NaN,164.11,NaN,NaN,NaN,176.26,NaN,NaN,NaN,NaN,NaN


We can also get row and column subtotals with the margins parameter. Let's count the number of times each station recorded snow per month and include the subtotals:

In [34]:
snow_data = weather.query('datatype == "SNOW"')
pd.crosstab(
    index=snow_data.station_name,
    columns=snow_data.index.month,
    colnames=['month'],
    values=snow_data.value,
    aggfunc=lambda x: (x > 0).sum(),
    margins=True, # show row and column subtotals
    margins_name='total observations of snow' # name the subtotals
)

month,1,2,3,4,5,6,7,8,9,10,11,12,total observations of snow
station_name,,,,,,,,,,,,,
"ALBERTSON 0.2 SSE, NY US",0.00,1.00,1.00,1.00,2.00,0.00,1.00,0.00,0.00,0.00,1.00,2.00,9
"AMITYVILLE 0.1 WSW, NY US",0.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,3
"AMITYVILLE 0.6 NNE, NY US",1.00,1.00,0.00,1.00,2.00,1.00,1.00,0.00,0.00,0.00,0.00,1.00,8
"ARMONK 0.3 SE, NY US",3.00,2.00,0.00,2.00,2.00,3.00,1.00,1.00,2.00,1.00,3.00,3.00,23
"BLOOMINGDALE 0.7 SSE, NJ US",0.00,1.00,2.00,0.00,1.00,1.00,0.00,1.00,0.00,1.00,0.00,1.00,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"WESTFIELD 0.6 NE, NJ US",1.00,1.00,2.00,0.00,0.00,2.00,NaN,0.00,1.00,2.00,0.00,0.00,9
"WOODBRIDGE TWP 1.1 ESE, NJ US",1.00,1.00,0.00,1.00,0.00,0.00,1.00,1.00,3.00,0.00,1.00,2.00,11
"WOODBRIDGE TWP 1.1 NNE, NJ US",2.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,3.00,0.00,1.00,0.00,7
